In [ ]:
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn.init as init

torch.manual_seed(3242023) # Seeded with a constant, so that behavior is deterministic.
torch.cuda.manual_seed_all(3242023)

torch.use_deterministic_algorithms(True)

if torch.cuda.is_available():
    device = "cuda"
else:
   device = "cpu"

print(f"Pytorch running on {device}")

In [ ]:
def _weights_init(m):
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)

class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, option='A'):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            if option == 'A':
                """
                For CIFAR10 ResNet paper uses option A.
                """
                self.shortcut = LambdaLayer(lambda x:
                                            F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
            elif option == 'B':
                self.shortcut = nn.Sequential(
                     nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                     nn.BatchNorm2d(self.expansion * planes)
                )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(64, num_classes)

        self.apply(_weights_init)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def resnet20():
    return ResNet(BasicBlock, [3, 3, 3]).to(device)

"""
All code above here with the exception of setting the random seed was taken from
https://github.com/akamaster/pytorch_resnet_cifar10/blob/master/resnet.py
"""

def pretrained_resnet20(pretrained_path):
    ans = ResNet(BasicBlock, [3, 3, 3]).to(device)
    ans.load_state_dict(torch.load(pretrained_path))
    return ans

In [ ]:
def get_params(model):
    params = []
    for param in model.parameters():
        params.append(param.view(-1))
    params = torch.cat(params)
    return params

def get_l2_distance(model1, model2):
    params1 = get_params(model1)
    params2 = get_params(model2)
    diff = params1-params2
    norm = torch.norm(diff, p=2)
    return norm

def get_l2_distance_as_fraction(model1, model2):
    params1 = get_params(model1)
    params2 = get_params(model2)
    diff = params1-params2
    norm = torch.norm(diff, p=2)
    size = (torch.norm(params1, p=2) + torch.norm(params2, p=2)) / 2
    return norm/size

In [ ]:
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD_DEV = (0.2023, 0.1994, 0.2010)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD_DEV),
])

samples = 5000
samples_to_remove = 0
epochs = 100

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainset_full = torch.utils.data.Subset(trainset, range(samples))
trainset_partial =  torch.utils.data.Subset(trainset, range(samples-samples_to_remove))

trainloader_full = torch.utils.data.DataLoader(trainset_full, batch_size=samples,
                                          shuffle=False)
trainloader_partial = torch.utils.data.DataLoader(trainset_partial, batch_size=(samples-samples_to_remove),
                                          shuffle=False)

net_full = resnet20()
net_partial = resnet20()
net_partial.load_state_dict(net_full.state_dict()) # Ensure we start from the same place with both models. This appears to be the crucial step.

loss_function = torch.nn.CrossEntropyLoss()

optimizer_full = torch.optim.SGD(net_full.parameters(), lr=0.1, momentum=0.9)
optimizer_partial = torch.optim.SGD(net_partial.parameters(), lr=0.1, momentum=0.9)


loss_by_epoch_full = []
loss_by_epoch_partial = []
epoch_wide_running_loss_full = 0
epoch_wide_running_loss_partial = 0

In [ ]:
02print("Beginning training...")

net_full.train()
net_partial.train()

for epoch in range(epochs):

    dist = get_l2_distance(net_full, net_partial)
    dist_as_fraction = get_l2_distance_as_fraction(net_full, net_partial)
    print(f"Before epoch {epoch}: L2 norm of difference after removing {samples_to_remove} samples out of {samples} sample dataset: {dist}. This is {dist_as_fraction*100}% of the l2 norm of the models themselves.")

    torch.manual_seed(3242023)
    torch.cuda.manual_seed_all(3242023)

    for i, data in enumerate(trainloader_full, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer_full.zero_grad()

        # forward + backward + optimize
        outputs = net_full(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer_full.step()

        # log statistics
        epoch_wide_running_loss_full += loss.item()
    loss_by_epoch_full.append(epoch_wide_running_loss_full/samples)

    torch.manual_seed(3242023)
    torch.cuda.manual_seed_all(3242023)

    for i, data in enumerate(trainloader_partial, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer_partial.zero_grad()

        # forward + backward + optimize
        outputs = net_partial(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer_partial.step()

        # log statistics
        epoch_wide_running_loss_partial += loss.item()
    loss_by_epoch_partial.append(epoch_wide_running_loss_partial/(samples-samples_to_remove))

    print(f"Epoch {epoch}: full dataset loss: {epoch_wide_running_loss_full/samples}, dataset sans {samples_to_remove} samples loss: {epoch_wide_running_loss_partial/(samples-samples_to_remove)}")
    epoch_wide_running_loss_full = 0
    epoch_wide_running_loss_partial = 0

print(loss_by_epoch_full, loss_by_epoch_partial)

In [ ]:
dist = get_l2_distance(net_full, net_partial)
dist_as_fraction = get_l2_distance_as_fraction(net_full, net_partial)
print(f"L2 norm of difference after removing {samples_to_remove} samples out of {samples} sample dataset: {dist}. This is {dist_as_fraction*100}% of the l2 norm of the models themselves.")